# Segmenting and Clustering Neighborhoods in Toronto  (Bourough:Scarborough)



In this analysis,I performed conversion of addresses into their equivalent latitude and longitude values. Moreover,learnt how to explore neighborhoods in Toronto City using the Foursquare API . Using the explore function to get the most common venue categories in each neighborhood, and then used this feature to group the neighborhoods into clusters. Used the k-means clustering algorithm to accomplish this task. Also used the Folium library to visualize the neighborhoods and their emerging clusters.

Table of Contents

1. Download and Explore Dataset
2. Explore Neighborhoods in Toronto ,Canada
3. Analyze Each Neighborhood
4. Cluster Neighborhoods
5. Examine Clusters

In [2]:
#Downloading the dependencies 

from bs4 import BeautifulSoup #library for web scraping
import requests  # library to handle requests
import json  # library to handle JSON files
import xml
import pandas as pd #Python library data manipulation and analysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


from geopy.geocoders import Nominatim # convert an address into latitude and longitude values


from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans
# !conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [13]:
from requests import get

url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

response = get(url)
print(response.text[:50])

<!DOCTYPE html>
<html class="client-nojs" lang="en


In [18]:
from bs4 import BeautifulSoup

html_soup = BeautifulSoup(response.text, 'lxml')
type(html_soup)

bs4.BeautifulSoup

In [21]:
html_soup = soup.find('table',{'class':'wikitable sortable'})
html_soup

<table class="wikitable sortable">
<tbody><tr>
<th>Postcode</th>
<th>Borough</th>
<th>Neighbourhood
</th></tr>
<tr>
<td>M1A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M2A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M3A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Parkwoods" title="Parkwoods">Parkwoods</a>
</td></tr>
<tr>
<td>M4A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Victoria_Village" title="Victoria Village">Victoria Village</a>
</td></tr>
<tr>
<td>M5A</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Harbourfront_(Toronto)" title="Harbourfront (Toronto)">Harbourfront</a>
</td></tr>
<tr>
<td>M5A</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Regent_Park" title="Regent Park">Regent Park</a>
</td></tr>
<tr>
<td>M6A</td>

In [26]:
code_containers = html_soup.find_all('tr')
print(type(code_containers))
print(len(code_containers))

<class 'bs4.element.ResultSet'>
290


In [225]:
#Removing first '<tr>' as it contians the column headers
code_containers_no_columnnames = code_containers[1:]
len(code_containers_no_columnnames)

289

In [66]:
test_list = []
for tr in code_containers_no_columnnames: #Iterating through rows and columns
    td=tr.find_all('td')
    for x in td:
#         print(x.text)
        test_list.append(x.text)
test_list

['M1A',
 'Not assigned',
 'Not assigned\n',
 'M2A',
 'Not assigned',
 'Not assigned\n',
 'M3A',
 'North York',
 'Parkwoods\n',
 'M4A',
 'North York',
 'Victoria Village\n',
 'M5A',
 'Downtown Toronto',
 'Harbourfront\n',
 'M5A',
 'Downtown Toronto',
 'Regent Park\n',
 'M6A',
 'North York',
 'Lawrence Heights\n',
 'M6A',
 'North York',
 'Lawrence Manor\n',
 'M7A',
 "Queen's Park",
 'Not assigned\n',
 'M8A',
 'Not assigned',
 'Not assigned\n',
 'M9A',
 'Etobicoke',
 'Islington Avenue\n',
 'M1B',
 'Scarborough',
 'Rouge\n',
 'M1B',
 'Scarborough',
 'Malvern\n',
 'M2B',
 'Not assigned',
 'Not assigned\n',
 'M3B',
 'North York',
 'Don Mills North\n',
 'M4B',
 'East York',
 'Woodbine Gardens\n',
 'M4B',
 'East York',
 'Parkview Hill\n',
 'M5B',
 'Downtown Toronto',
 'Ryerson\n',
 'M5B',
 'Downtown Toronto',
 'Garden District\n',
 'M6B',
 'North York',
 'Glencairn\n',
 'M7B',
 'Not assigned',
 'Not assigned\n',
 'M8B',
 'Not assigned',
 'Not assigned\n',
 'M9B',
 'Etobicoke',
 'Cloverdale\n',

In [99]:
#Seperating postalcodes
PostalCode = []

for a in test_list[::3]:
    PostalCode.append(a)

In [100]:
#Seperating Borough
Borough = []

for a in test_list[1::3]:
    Borough.append(a)

In [101]:
#Seperating neighborhood
Neighborhood = []

for a in test_list[2::3]:
    Neighborhood.append(a)

In [102]:
print(PostalCode[:10])
print(Borough[:10])
print(Neighborhood[:10])

['M1A', 'M2A', 'M3A', 'M4A', 'M5A', 'M5A', 'M6A', 'M6A', 'M7A', 'M8A']
['Not assigned', 'Not assigned', 'North York', 'North York', 'Downtown Toronto', 'Downtown Toronto', 'North York', 'North York', "Queen's Park", 'Not assigned']
['Not assigned\n', 'Not assigned\n', 'Parkwoods\n', 'Victoria Village\n', 'Harbourfront\n', 'Regent Park\n', 'Lawrence Heights\n', 'Lawrence Manor\n', 'Not assigned\n', 'Not assigned\n']


In [103]:
print(len(PostalCode))
print(len(Borough))
print(len(Neighborhood))

289
289
289


In [104]:
# Stripping the '\n' in the nieghborhood values
Neighborhood = [i.strip() for i in Neighborhood]
print(Neighborhood)

['Not assigned', 'Not assigned', 'Parkwoods', 'Victoria Village', 'Harbourfront', 'Regent Park', 'Lawrence Heights', 'Lawrence Manor', 'Not assigned', 'Not assigned', 'Islington Avenue', 'Rouge', 'Malvern', 'Not assigned', 'Don Mills North', 'Woodbine Gardens', 'Parkview Hill', 'Ryerson', 'Garden District', 'Glencairn', 'Not assigned', 'Not assigned', 'Cloverdale', 'Islington', 'Martin Grove', 'Princess Gardens', 'West Deane Park', 'Highland Creek', 'Rouge Hill', 'Port Union', 'Not assigned', 'Flemingdon Park', 'Don Mills South', 'Woodbine Heights', 'St. James Town', 'Humewood-Cedarvale', 'Not assigned', 'Not assigned', 'Bloordale Gardens', 'Eringate', 'Markland Wood', 'Old Burnhamthorpe', 'Guildwood', 'Morningside', 'West Hill', 'Not assigned', 'Not assigned', 'The Beaches', 'Berczy Park', 'Caledonia-Fairbanks', 'Not assigned', 'Not assigned', 'Not assigned', 'Woburn', 'Not assigned', 'Not assigned', 'Leaside', 'Central Bay Street', 'Christie', 'Not assigned', 'Not assigned', 'Not ass

In [108]:
#Constructing the main dataframe 
main_df=pd.DataFrame({'PostalCode':PostalCode,
                 'Borough':Borough,
                 'Neighborhood':Neighborhood})

In [109]:
#Fixing the order of the columns in the main dataframe
main_df=main_df[['PostalCode','Borough','Neighborhood']]

In [110]:
#Visualizing the first few rows of the dataframe
main_df.head(10)

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
9,M8A,Not assigned,Not assigned


In [111]:
len(main_df['PostalCode'])

289

In [129]:
#Checking for no of 'Not assigned' values in Neighborhood column
len(main_df[main_df['Neighborhood'].str.contains('Not assigned')])

78

In [130]:
#Checking for no of 'Not assigned' values in Borough column
len(main_df[main_df['Borough'].str.contains('Not assigned')])

77

In [161]:
#Removing rows with 'Not assigned' values in the final dataframe
main_df1 = main_df[(main_df['Borough'] != 'Not assigned') & (main_df['Neighborhood'] != 'Not assigned')]
len(main_df1)

211

In [162]:
#Visualizing the first few rows of the dataframe
main_df1.rename(columns = {'PostalCode':'Postal Code'}, inplace=True)
main_df1 = main_df1.set_index('Postal Code')
main_df1.head(10)

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/pandas/core/frame.py:3027: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


,Borough,Neighborhood
Postal Code,,
M3A,North York,Parkwoods
M4A,North York,Victoria Village
M5A,Downtown Toronto,Harbourfront
M5A,Downtown Toronto,Regent Park
M6A,North York,Lawrence Heights
M6A,North York,Lawrence Manor
M9A,Etobicoke,Islington Avenue
M1B,Scarborough,Rouge
M1B,Scarborough,Malvern


In [163]:
type(main_df1)

pandas.core.frame.DataFrame

In [164]:
main_df1.shape

(211, 2)

We are supposed to use the Google Maps Geocoding API to get the latitude and the longitude coordinates of each neighborhood. However, this is a paid service API: http://geoawesomeness.com/developers-up-in-arms-over-google-maps-api-insane-price-hike/, so we will use the Geocoder Python package instead: https://geocoder.readthedocs.io/index.html.

In [158]:
toronto_geocsv = 'https://cocl.us/Geospatial_data'
!wget -q -O 'toronto_m.geospatial_data.csv' toronto_geocsv
geocsv_data = pd.read_csv(toronto_geocsv).set_index("Postal Code")
geocsv_data.head()

,Latitude,Longitude
Postal Code,,
M1B,43.806686,-79.194353
M1C,43.784535,-79.160497
M1E,43.763573,-79.188711
M1G,43.770992,-79.216917
M1H,43.773136,-79.239476


In [165]:
#Combining Two data frames with mapped postcodes
toronto_neighborhoods = main_df1.join(geocsv_data)
toronto_neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
Postal Code,,,,
M1B,Scarborough,Rouge,43.806686,-79.194353
M1B,Scarborough,Malvern,43.806686,-79.194353
M1C,Scarborough,Highland Creek,43.784535,-79.160497
M1C,Scarborough,Rouge Hill,43.784535,-79.160497
M1C,Scarborough,Port Union,43.784535,-79.160497


In [166]:
toronto_neighborhoods.shape

(211, 4)

## Use geopy library to get the latitude and longitude values of Toronto

In [183]:
address = 'Toronto, Ontario Canada'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto Canada are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto Canada are 43.653963, -79.387207.


## Create a map of Toronto with neighborhoods superimposed on top.

In [184]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_neighborhoods['Latitude'], toronto_neighborhoods['Longitude'], toronto_neighborhoods['Borough'], toronto_neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=4,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#87cefa',
        fill_opacity=0.5,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [179]:
toronto_neighborhoods.reset_index(level=0, inplace=True)
toronto_neighborhoods.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,Rouge,43.806686,-79.194353
1,M1B,Scarborough,Malvern,43.806686,-79.194353
2,M1C,Scarborough,Highland Creek,43.784535,-79.160497
3,M1C,Scarborough,Rouge Hill,43.784535,-79.160497
4,M1C,Scarborough,Port Union,43.784535,-79.160497


However, for illustration purposes, let's simplify the above map and segment and cluster only the neighborhoods in Scarborough. So let's slice the original dataframe and create a new dataframe of the Scarborough data.

In [180]:
Scarborough_data = toronto_neighborhoods[toronto_neighborhoods['Borough'] == 'Scarborough'].reset_index(drop=True)
Scarborough_data

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,Rouge,43.806686,-79.194353
1,M1B,Scarborough,Malvern,43.806686,-79.194353
2,M1C,Scarborough,Highland Creek,43.784535,-79.160497
3,M1C,Scarborough,Rouge Hill,43.784535,-79.160497
4,M1C,Scarborough,Port Union,43.784535,-79.160497
5,M1E,Scarborough,Guildwood,43.763573,-79.188711
6,M1E,Scarborough,Morningside,43.763573,-79.188711
7,M1E,Scarborough,West Hill,43.763573,-79.188711
8,M1G,Scarborough,Woburn,43.770992,-79.216917
9,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


## Geographical coordinates of Scarborough

In [181]:
address = 'Scarborough,Canada'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude_s = location.latitude
longitude_s = location.longitude
print('The geograpical coordinate of Scarborough are {}, {}.'.format(latitude_s, longitude_s))

The geograpical coordinate of Scarborough are 43.773077, -79.257774.


#### Neighborhoods in Scarborough

In [182]:
# create map of Scarborough using latitude and longitude values
map_scarborough = folium.Map(location=[latitude_s, longitude_s], zoom_start=10)

# add markers to map
for lat, lng, bor, nei in zip(Scarborough_data['Latitude'], Scarborough_data['Longitude'],Scarborough_data['Borough'], Scarborough_data['Neighborhood']):
    
    label = '{}, {}'.format(nei, bor)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_scarborough)  
    
map_scarborough

## Foursquare Credentials and Version

In [185]:
CLIENT_ID = 'P3BMDGVCXWMB23R12AS2CY1MD0BHDMQLUVIVN5A51BMVSD5C' # your Foursquare ID
CLIENT_SECRET = 'WOEBXTQIF54HHOF3M5OHKW4VIIYED0XYYIWEB54PTA2NQYRW' # your Foursquare Secret
VERSION = '20190217' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: P3BMDGVCXWMB23R12AS2CY1MD0BHDMQLUVIVN5A51BMVSD5C
CLIENT_SECRET:WOEBXTQIF54HHOF3M5OHKW4VIIYED0XYYIWEB54PTA2NQYRW


### Exploring the first neighborhood of Scarborough

In [186]:
#Name of the neighborhood
first_nei=Scarborough_data['Neighborhood'][0]
first_nei

'Rouge'

In [187]:
#Location of the neighborhood
first_nei_lat=Scarborough_data.loc[0,'Latitude']
first_nei_lon=Scarborough_data.loc[0,'Longitude']
print('Latitude and longitude values of {} are {}, {}.'.format(first_nei, 
                                                               first_nei_lat, 
                                                               first_nei_lon))

Latitude and longitude values of Rouge are 43.806686299999996, -79.19435340000001.


Now, let's get the top 100 venues that are in Malvern within a radius of 500 meters.

In [189]:
radius = 500 
LIMIT = 100
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    first_nei_lat, 
    first_nei_lon, 
    radius, 
    LIMIT)

url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=P3BMDGVCXWMB23R12AS2CY1MD0BHDMQLUVIVN5A51BMVSD5C&client_secret=WOEBXTQIF54HHOF3M5OHKW4VIIYED0XYYIWEB54PTA2NQYRW&v=20190217&ll=43.806686299999996,-79.19435340000001&radius=500&limit=100'

In [190]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5c6bb7781ed2196e47d341da'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-4bb6b9446edc76b0d771311c-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/fastfood_',
          'suffix': '.png'},
         'id': '4bf58dd8d48988d16e941735',
         'name': 'Fast Food Restaurant',
         'pluralName': 'Fast Food Restaurants',
         'primary': True,
         'shortName': 'Fast Food'}],
       'id': '4bb6b9446edc76b0d771311c',
       'location': {'cc': 'CA',
        'city': 'Toronto',
        'country': 'Canada',
        'crossStreet': 'Morningside & Sheppard',
        'distance': 387,
        'formattedAddress': ['Toronto ON', 'Canada'],
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.80744841934756,
          'ln

#### Since we havent received much results in this neighborhood lets select a different neighborhood 


In [191]:
second_nei_name=Scarborough_data.loc[15,'Neighborhood']
second_nei_lat=Scarborough_data.loc[15,'Latitude']
second_nei_lon=Scarborough_data.loc[15,'Longitude']
print('Latitude and longitude values of {} are {}, {}.'.format(second_nei_name, 
                                                               second_nei_lat, 
                                                               second_nei_lon))

Latitude and longitude values of Golden Mile are 43.711111700000004, -79.2845772.


In [193]:
radius = 500 
LIMIT = 100
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    second_nei_lat, 
    second_nei_lon, 
    radius, 
    LIMIT)

url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=P3BMDGVCXWMB23R12AS2CY1MD0BHDMQLUVIVN5A51BMVSD5C&client_secret=WOEBXTQIF54HHOF3M5OHKW4VIIYED0XYYIWEB54PTA2NQYRW&v=20190217&ll=43.711111700000004,-79.2845772&radius=500&limit=100'

In [194]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5c6bb7fef594df76b63e5f4e'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-4bc8e605b6c49c740e5b8d91-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/fastfood_',
          'suffix': '.png'},
         'id': '4bf58dd8d48988d16e941735',
         'name': 'Fast Food Restaurant',
         'pluralName': 'Fast Food Restaurants',
         'primary': True,
         'shortName': 'Fast Food'}],
       'id': '4bc8e605b6c49c740e5b8d91',
       'location': {'address': '625 Pharmacy Avenue',
        'cc': 'CA',
        'city': 'Toronto',
        'country': 'Canada',
        'distance': 499,
        'formattedAddress': ['625 Pharmacy Avenue',
         'Toronto ON M1L 3H3',
         'Canada'],
        'labeledLatLngs': [{'label': 'display',
       

In [195]:
results['response']['groups'][0]['items'][0]['venue']['categories'][0]['name']

'Fast Food Restaurant'

### All the information is in the items key. Lets create a get_category_type function .

In [196]:
venues=results['response']['groups'][0]['items']
nearby_venues = json_normalize(venues)
nearby_venues.columns

Index(['reasons.count', 'reasons.items', 'referralId', 'venue.categories',
       'venue.id', 'venue.location.address', 'venue.location.cc',
       'venue.location.city', 'venue.location.country',
       'venue.location.crossStreet', 'venue.location.distance',
       'venue.location.formattedAddress', 'venue.location.labeledLatLngs',
       'venue.location.lat', 'venue.location.lng', 'venue.location.postalCode',
       'venue.location.state', 'venue.name', 'venue.photos.count',
       'venue.photos.groups'],
      dtype='object')

In [197]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [198]:
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]
nearby_venues

,venue.name,venue.categories,venue.location.lat,venue.location.lng
0,Dairy Queen,[{'icon': {'prefix': 'https://ss3.4sqi.net/img...,43.710378,-79.290701
1,TTC Bus #68 Warden,[{'icon': {'prefix': 'https://ss3.4sqi.net/img...,43.711778,-79.279714
2,TTC Bus 102 Markham Road,[{'icon': {'prefix': 'https://ss3.4sqi.net/img...,43.711381,-79.279588
3,Warden Station Bus Loop,[{'icon': {'prefix': 'https://ss3.4sqi.net/img...,43.711241,-79.279576
4,Bakery On The Go,[{'icon': {'prefix': 'https://ss3.4sqi.net/img...,43.711271,-79.279506
5,Cafe on the go,[{'icon': {'prefix': 'https://ss3.4sqi.net/img...,43.711151,-79.279469
6,Star Grazing,[{'icon': {'prefix': 'https://ss3.4sqi.net/img...,43.711663,-79.279466
7,Warden Woods Park,[{'icon': {'prefix': 'https://ss3.4sqi.net/img...,43.710527,-79.278966
8,Edward Jones - Financial Advisor: Thomas Moxam,[{'icon': {'prefix': 'https://ss3.4sqi.net/img...,43.714543,-79.281304
9,Clairlea Futbol Centre,[{'icon': {'prefix': 'https://ss3.4sqi.net/img...,43.715234,-79.286506


In [199]:
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues

,name,categories,lat,lng
0,Dairy Queen,Fast Food Restaurant,43.710378,-79.290701
1,TTC Bus #68 Warden,Bus Line,43.711778,-79.279714
2,TTC Bus 102 Markham Road,Bus Line,43.711381,-79.279588
3,Warden Station Bus Loop,Bus Station,43.711241,-79.279576
4,Bakery On The Go,Bakery,43.711271,-79.279506
5,Cafe on the go,Bakery,43.711151,-79.279469
6,Star Grazing,Food Truck,43.711663,-79.279466
7,Warden Woods Park,Park,43.710527,-79.278966
8,Edward Jones - Financial Advisor: Thomas Moxam,Business Service,43.714543,-79.281304
9,Clairlea Futbol Centre,Soccer Field,43.715234,-79.286506


# Explore Neighborhoods in Scarborough
### Let's create a function to repeat the same process to all the neighborhoods in Scarborough

In [200]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        venue_results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in venue_results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

### Following script run the above function on each neighborhood and create a new dataframe called scarborough_venues.

In [201]:
scarborough_venues = getNearbyVenues(names=Scarborough_data['Neighborhood'],
                                   latitudes=Scarborough_data['Latitude'],
                                   longitudes=Scarborough_data['Longitude']
                                  )



Rouge
Malvern
Highland Creek
Rouge Hill
Port Union
Guildwood
Morningside
West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park
Ionview
Kennedy Park
Clairlea
Golden Mile
Oakridge
Cliffcrest
Cliffside
Scarborough Village West
Birch Cliff
Cliffside West
Dorset Park
Scarborough Town Centre
Wexford Heights
Maryvale
Wexford
Agincourt
Clarks Corners
Sullivan
Tam O'Shanter
Agincourt North
L'Amoreaux East
Milliken
Steeles East
L'Amoreaux West
Steeles West
Upper Rouge


In [202]:
print(scarborough_venues.shape)
scarborough_venues.head()

(195, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Rouge,43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,Malvern,43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
2,Highland Creek,43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
3,Highland Creek,43.784535,-79.160497,Affordable Toronto Movers,43.787919,-79.162977,Moving Target
4,Rouge Hill,43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar


### Venues per neighborhood

In [203]:
scarborough_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,5,5,5,5,5,5
Agincourt North,2,2,2,2,2,2
Birch Cliff,4,4,4,4,4,4
Cedarbrae,7,7,7,7,7,7
Clairlea,10,10,10,10,10,10
Clarks Corners,9,9,9,9,9,9
Cliffcrest,2,2,2,2,2,2
Cliffside,2,2,2,2,2,2
Cliffside West,4,4,4,4,4,4


### Unique Categories

In [204]:

print('There are {} uniques categories.'.format(len(scarborough_venues['Venue Category'].unique())))

There are 51 uniques categories.


# Analyze Each Neighborhood

In [205]:
# one hot encoding
scarborough_onehot = pd.get_dummies(scarborough_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
scarborough_onehot['Neighborhood'] = scarborough_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [scarborough_onehot.columns[-1]] + list(scarborough_onehot.columns[:-1])
scarborough_onehot = scarborough_onehot[fixed_columns]
scarborough_onehot.head()


,Neighborhood,American Restaurant,Athletics & Sports,Auto Garage,Bakery,Bank,Bar,Breakfast Spot,Bus Line,Bus Station,Business Service,Café,Caribbean Restaurant,Chinese Restaurant,Coffee Shop,College Stadium,Department Store,Discount Store,Electronics Store,Fast Food Restaurant,Food Truck,Fried Chicken Joint,General Entertainment,Grocery Store,Hakka Restaurant,Hobby Shop,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Korean Restaurant,Latin American Restaurant,Lounge,Medical Center,Mexican Restaurant,Middle Eastern Restaurant,Motel,Moving Target,Noodle House,Park,Pet Store,Pharmacy,Pizza Place,Playground,Rental Car Location,Sandwich Place,Skating Rink,Smoke Shop,Soccer Field,Thai Restaurant,Thrift / Vintage Store,Train Station,Vietnamese Restaurant
0,Rouge,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Malvern,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Highland Creek,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Highland Creek,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Rouge Hill,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [206]:
scarborough_onehot.shape   #90 venues and 56 unique categories

(195, 52)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category¶

In [207]:
scarborough_grouped = scarborough_onehot.groupby('Neighborhood').mean().reset_index()
scarborough_grouped

,Neighborhood,American Restaurant,Athletics & Sports,Auto Garage,Bakery,Bank,Bar,Breakfast Spot,Bus Line,Bus Station,Business Service,Café,Caribbean Restaurant,Chinese Restaurant,Coffee Shop,College Stadium,Department Store,Discount Store,Electronics Store,Fast Food Restaurant,Food Truck,Fried Chicken Joint,General Entertainment,Grocery Store,Hakka Restaurant,Hobby Shop,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Korean Restaurant,Latin American Restaurant,Lounge,Medical Center,Mexican Restaurant,Middle Eastern Restaurant,Motel,Moving Target,Noodle House,Park,Pet Store,Pharmacy,Pizza Place,Playground,Rental Car Location,Sandwich Place,Skating Rink,Smoke Shop,Soccer Field,Thai Restaurant,Thrift / Vintage Store,Train Station,Vietnamese Restaurant
0,Agincourt,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.200000,0.0,0.0,0.0,0.00,0.000000,0.200000,0.000000,0.00,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.00,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.2,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.200000,0.20,0.000000,0.0,0.000000,0.000000,0.0,0.000000
1,Agincourt North,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.00,0.000000,0.000000,0.000000,0.00,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.00,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.5,0.000000,0.000000,0.000000,0.5,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.0,0.000000
2,Birch Cliff,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.25,0.000000,0.000000,0.000000,0.25,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.25,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.25,0.000000,0.0,0.000000,0.000000,0.0,0.000000
3,Cedarbrae,0.0,0.142857,0.000000,0.142857,0.142857,0.0,0.000000,0.0,0.0,0.0,0.00,0.142857,0.000000,0.000000,0.00,0.0,0.0,0.000000,0.000000,0.0,0.142857,0.00,0.000000,0.142857,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.0,0.142857,0.000000,0.0,0.000000
4,Clairlea,0.0,0.000000,0.000000,0.200000,0.000000,0.0,0.000000,0.2,0.1,0.1,0.00,0.000000,0.000000,0.000000,0.00,0.0,0.0,0.000000,0.100000,0.1,0.000000,0.00,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.1,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.1,0.000000,0.000000,0.0,0.000000
5,Clarks Corners,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.00,0.000000,0.111111,0.000000,0.00,0.0,0.0,0.000000,0.111111,0.0,0.111111,0.00,0.000000,0.000000,0.0,0.000000,0.111111,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.111111,0.0,0.000000,0.111111,0.222222,0.0,0.000000,0.000000,0.00,0.000000,0.0,0.111111,0.000000,0.0,0.000000
6,Cliffcrest,0.5,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.00,0.000000,0.000000,0.000000,0.00,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.00,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.5,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.0,0.000000
7,Cliffside,0.5,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.00,0.000000,0.000000,0.000000,0.00,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.00,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.5,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.0,0.000000
8,Cliffside West,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.25,0.000000,0.000000,0.000000,0.25,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.25,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00000

In [208]:
scarborough_grouped.shape

(37, 52)

### Each neighborhood along with the top 5 most common venues

In [209]:
num_top_venues = 5
for hood in scarborough_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp =scarborough_grouped[scarborough_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                venue  freq
0              Lounge   0.2
1      Breakfast Spot   0.2
2        Skating Rink   0.2
3      Sandwich Place   0.2
4  Chinese Restaurant   0.2


----Agincourt North----
                 venue  freq
0           Playground   0.5
1                 Park   0.5
2  American Restaurant   0.0
3            Pet Store   0.0
4    Korean Restaurant   0.0


----Birch Cliff----
                   venue  freq
0  General Entertainment  0.25
1           Skating Rink  0.25
2                   Café  0.25
3        College Stadium  0.25
4    American Restaurant  0.00


----Cedarbrae----
                venue  freq
0              Bakery  0.14
1                Bank  0.14
2  Athletics & Sports  0.14
3     Thai Restaurant  0.14
4    Hakka Restaurant  0.14


----Clairlea----
              venue  freq
0            Bakery   0.2
1          Bus Line   0.2
2      Soccer Field   0.1
3       Bus Station   0.1
4  Business Service   0.1


----Clarks Corners----
                  

### Function to sort the venues in descending order.

In [210]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [211]:
import numpy as np
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = scarborough_grouped['Neighborhood']

for ind in np.arange(scarborough_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(scarborough_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Chinese Restaurant,Skating Rink,Sandwich Place,Breakfast Spot,Lounge,Vietnamese Restaurant,General Entertainment,Fried Chicken Joint,Food Truck,Fast Food Restaurant
1,Agincourt North,Park,Playground,Caribbean Restaurant,General Entertainment,Fried Chicken Joint,Food Truck,Fast Food Restaurant,Electronics Store,Discount Store,Department Store
2,Birch Cliff,General Entertainment,Skating Rink,College Stadium,Café,Vietnamese Restaurant,Chinese Restaurant,Grocery Store,Fried Chicken Joint,Food Truck,Fast Food Restaurant
3,Cedarbrae,Caribbean Restaurant,Bakery,Fried Chicken Joint,Bank,Hakka Restaurant,Athletics & Sports,Thai Restaurant,Auto Garage,College Stadium,Grocery Store
4,Clairlea,Bakery,Bus Line,Park,Soccer Field,Food Truck,Fast Food Restaurant,Bus Station,Business Service,Coffee Shop,General Entertainment
5,Clarks Corners,Pizza Place,Noodle House,Fried Chicken Joint,Thai Restaurant,Italian Restaurant,Fast Food Restaurant,Pharmacy,Chinese Restaurant,Caribbean Restaurant,Food Truck
6,Cliffcrest,American Restaurant,Motel,Hobby Shop,Grocery Store,General Entertainment,Fried Chicken Joint,Food Truck,Fast Food Restaurant,Electronics Store,Discount Store
7,Cliffside,American Restaurant,Motel,Hobby Shop,Grocery Store,General Entertainment,Fried Chicken Joint,Food Truck,Fast Food Restaurant,Electronics Store,Discount Store
8,Cliffside West,General Entertainment,Skating Rink,College Stadium,Café,Vietnamese Restaurant,Chinese Restaurant,Grocery Store,Fried Chicken Joint,Food Truck,Fast Food Restaurant
9,Dorset Park,Indian Restaurant,Pet Store,Chinese Restaurant,Latin American Restaurant,Vietnamese Restaurant,Thrift / Vintage Store,Smoke Shop,Caribbean Restaurant,Fried Chicken Joint,Food Truck


# Cluster Neighborhoods

In [217]:
# set number of clusters
# Using k-means to cluster the neighborhood into 4 clusters.
kclusters = 4

scarborough_grouped_clustering = scarborough_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(scarborough_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 2, 0, 1, 0, 2, 0, 0,
       3, 1, 3, 0, 2, 0, 2, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

Creating a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

Since the neighborhood 'Upper Rouge' has no venue data returned lets remove that neighborhood

In [219]:
scarborough_merged = Scarborough_data[0:37]

# add clustering labels
scarborough_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
scarborough_merged = scarborough_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

scarborough_merged.head() # check the last columns!

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,Rouge,43.806686,-79.194353,0,Fast Food Restaurant,Vietnamese Restaurant,Chinese Restaurant,Grocery Store,General Entertainment,Fried Chicken Joint,Food Truck,Electronics Store,Discount Store,Department Store
1,M1B,Scarborough,Malvern,43.806686,-79.194353,2,Fast Food Restaurant,Vietnamese Restaurant,Chinese Restaurant,Grocery Store,General Entertainment,Fried Chicken Joint,Food Truck,Electronics Store,Discount Store,Department Store
2,M1C,Scarborough,Highland Creek,43.784535,-79.160497,0,Bar,Moving Target,Vietnamese Restaurant,Coffee Shop,Grocery Store,General Entertainment,Fried Chicken Joint,Food Truck,Fast Food Restaurant,Electronics Store
3,M1C,Scarborough,Rouge Hill,43.784535,-79.160497,0,Bar,Moving Target,Vietnamese Restaurant,Coffee Shop,Grocery Store,General Entertainment,Fried Chicken Joint,Food Truck,Fast Food Restaurant,Electronics Store
4,M1C,Scarborough,Port Union,43.784535,-79.160497,0,Bar,Moving Target,Vietnamese Restaurant,Coffee Shop,Grocery Store,General Entertainment,Fried Chicken Joint,Food Truck,Fast Food Restaurant,Electronics Store


### Visualising the clusters

In [220]:
# create map
map_clusters = folium.Map(location=[latitude_s, longitude_s], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
colors_array = cm.rainbow(np.linspace(0, 1, kclusters))
rainbow = [colors.rgb2hex(i) for i in colors_array]
print(rainbow)
# add markers to the map
markers_colors = []
for lat, lon, nei , cluster in zip(scarborough_merged['Latitude'], scarborough_merged['Longitude'], scarborough_merged['Neighborhood'], scarborough_merged['Cluster Labels']):
    label = folium.Popup(str(nei) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

['#8000ff', '#2adddd', '#d4dd80', '#ff0000']


## Examine Clusters
#### Examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, we can then assign a name to each cluster.



#### Cluster 1

In [221]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 0,scarborough_merged.columns[[2] + list(range(5, scarborough_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Rouge,0,Fast Food Restaurant,Vietnamese Restaurant,Chinese Restaurant,Grocery Store,General Entertainment,Fried Chicken Joint,Food Truck,Electronics Store,Discount Store,Department Store
2,Highland Creek,0,Bar,Moving Target,Vietnamese Restaurant,Coffee Shop,Grocery Store,General Entertainment,Fried Chicken Joint,Food Truck,Fast Food Restaurant,Electronics Store
3,Rouge Hill,0,Bar,Moving Target,Vietnamese Restaurant,Coffee Shop,Grocery Store,General Entertainment,Fried Chicken Joint,Food Truck,Fast Food Restaurant,Electronics Store
4,Port Union,0,Bar,Moving Target,Vietnamese Restaurant,Coffee Shop,Grocery Store,General Entertainment,Fried Chicken Joint,Food Truck,Fast Food Restaurant,Electronics Store
5,Guildwood,0,Electronics Store,Breakfast Spot,Rental Car Location,Pizza Place,Medical Center,Mexican Restaurant,Vietnamese Restaurant,Chinese Restaurant,Fried Chicken Joint,Food Truck
6,Morningside,0,Electronics Store,Breakfast Spot,Rental Car Location,Pizza Place,Medical Center,Mexican Restaurant,Vietnamese Restaurant,Chinese Restaurant,Fried Chicken Joint,Food Truck
7,West Hill,0,Electronics Store,Breakfast Spot,Rental Car Location,Pizza Place,Medical Center,Mexican Restaurant,Vietnamese Restaurant,Chinese Restaurant,Fried Chicken Joint,Food Truck
8,Woburn,0,Coffee Shop,Korean Restaurant,Vietnamese Restaurant,Hobby Shop,Grocery Store,General Entertainment,Fried Chicken Joint,Food Truck,Fast Food Restaurant,Electronics Store
9,Cedarbrae,0,Caribbean Restaurant,Bakery,Fried Chicken Joint,Bank,Hakka Restaurant,Athletics & Sports,Thai Restaurant,Auto Garage,College Stadium,Grocery Store
10,Scarborough Village,0,Playground,Vietnamese Restaurant,Chinese Restaurant,Grocery Store,General Entertainment,Fried Chicken Joint,Food Truck,Fast Food Restaurant,Electronics Store,Discount Store


#### Cluster 2

In [222]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 1,scarborough_merged.columns[[2] + list(range(5, scarborough_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
18,Cliffside,1,American Restaurant,Motel,Hobby Shop,Grocery Store,General Entertainment,Fried Chicken Joint,Food Truck,Fast Food Restaurant,Electronics Store,Discount Store
24,Wexford Heights,1,Indian Restaurant,Pet Store,Chinese Restaurant,Latin American Restaurant,Vietnamese Restaurant,Thrift / Vintage Store,Smoke Shop,Caribbean Restaurant,Fried Chicken Joint,Food Truck


#### Cluster 3

In [223]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 2,scarborough_merged.columns[[2] + list(range(5, scarborough_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Malvern,2,Fast Food Restaurant,Vietnamese Restaurant,Chinese Restaurant,Grocery Store,General Entertainment,Fried Chicken Joint,Food Truck,Electronics Store,Discount Store,Department Store
16,Oakridge,2,Bakery,Bus Line,Park,Soccer Field,Food Truck,Fast Food Restaurant,Bus Station,Business Service,Coffee Shop,General Entertainment
20,Birch Cliff,2,General Entertainment,Skating Rink,College Stadium,Café,Vietnamese Restaurant,Chinese Restaurant,Grocery Store,Fried Chicken Joint,Food Truck,Fast Food Restaurant
27,Agincourt,2,Chinese Restaurant,Skating Rink,Sandwich Place,Breakfast Spot,Lounge,Vietnamese Restaurant,General Entertainment,Fried Chicken Joint,Food Truck,Fast Food Restaurant
29,Sullivan,2,Pizza Place,Noodle House,Fried Chicken Joint,Thai Restaurant,Italian Restaurant,Fast Food Restaurant,Pharmacy,Chinese Restaurant,Caribbean Restaurant,Food Truck


#### Cluster 4

In [224]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 3,scarborough_merged.columns[[2] + list(range(5, scarborough_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
13,Kennedy Park,3,Train Station,Hobby Shop,Discount Store,Department Store,Coffee Shop,Hakka Restaurant,Grocery Store,General Entertainment,Fried Chicken Joint,Food Truck
23,Scarborough Town Centre,3,Indian Restaurant,Pet Store,Chinese Restaurant,Latin American Restaurant,Vietnamese Restaurant,Thrift / Vintage Store,Smoke Shop,Caribbean Restaurant,Fried Chicken Joint,Food Truck
25,Maryvale,3,Middle Eastern Restaurant,Auto Garage,Smoke Shop,Sandwich Place,Breakfast Spot,Vietnamese Restaurant,Coffee Shop,General Entertainment,Fried Chicken Joint,Food Truck
